# Tweepy

This project plays around with tweepy to gather information from twitter.

##### Friend Maker
The following uses tweepy to follow friends of specified users according to certain parameters.

In [10]:
import tweepy
import webbrowser
import time
import pandas as pd
from datetime import datetime

In [11]:
key = "###################"
secret = "####################"

In [41]:
consumer_key = key
consumer_secret = secret
callback_uri = 'oob'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_uri)
redirect_url = auth.get_authorization_url()
print(redirect_url)
webbrowser.open(redirect_url)
user_pin_input = input("What's the pin value? ")

# output blocked for privacy.
# user would enter the value from the authorization website provided in output

In [42]:
auth.get_access_token(user_pin_input)
#output blocked

In [14]:
api = tweepy.API(auth)

In [15]:
me = api.me()

In [16]:
print(me.screen_name)

LukeMarcusBarry


In [20]:
user = api.get_user("mcuban")

In [21]:
print(user.followers_count, user.friends_count)

8063687 1670


In [22]:
my_new_friends = []
user_friends = user.friends()
for friend in user_friends:
    if friend.followers_count > 300 and friend.friends_count < 700:
        print(friend.screen_name)
        my_new_friends.append(friend.screen_name)
        relationship_ = api.create_friendship(friend.screen_name)

defipulse
BanklessHQ
flow_blockchain
dapperlabs
amadougallofall
UniswapProtocol
TheAmandaGorman
dipp
unchainedcap
Stanford_JBLP
HarvardLawBFI


In [23]:
to_remove = my_new_friends
print(my_new_friends)

['defipulse', 'BanklessHQ', 'flow_blockchain', 'dapperlabs', 'amadougallofall', 'UniswapProtocol', 'TheAmandaGorman', 'dipp', 'unchainedcap', 'Stanford_JBLP', 'HarvardLawBFI']


In [24]:
for username in to_remove:
    api.destroy_friendship(username)

##### Status updater

In [25]:
new_status = api.update_status("Hello world. Trying out updating my status using python.")

In [40]:
# new_status.destroy()

In [27]:
dir(new_status)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api',
 '_json',
 'author',
 'contributors',
 'coordinates',
 'created_at',
 'destroy',
 'entities',
 'favorite',
 'favorite_count',
 'favorited',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'parse',
 'parse_list',
 'place',
 'retweet',
 'retweet_count',
 'retweeted',
 'retweets',
 'source',
 'source_url',
 'text',
 'truncated',
 'user']

##### Timeline fetcher
The following creates a dataframe out of my own timeline and creates a function that grabs the timeline of any specified user and turns it into a dataframe.

In [30]:
my_timeline = api.home_timeline()
# print(my_timeline)

In [31]:
columns = set()
allowed_types = [str, int]
tweets_data = []

for status in my_timeline:
    print(status.author.screen_name)
    #print(status.text)
    status_dict = dict(vars(status))
    keys = vars(status).keys()
    single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
    for k in keys:
        try:
            v_type = type(status_dict[k])
        except:
            v_type = None
        if v_type != None:
            if v_type in allowed_types:
                single_tweet_data[k] = status_dict[k]
                columns.add(k)
    tweets_data.append(single_tweet_data)
    
header_columns = list(columns)
header_columns.append("user")
header_columns.append("author")

lexfridman
CaptainNasdaq
pcgamer
techreview
CNNPolitics
pcgamer
statedeptspox
TheBabylonBee
politico
MyNintendoNews
Frost_Giant
techreview
pcgamer
techreview
techreview
techreview
redditinvestors
cleantechnica
SecBlinken


In [32]:
df = pd.DataFrame(tweets_data, columns=header_columns)
df.head()

,favorite_count,text,retweet_count,quoted_status_id,source_url,lang,id,source,id_str,quoted_status_id_str,user,author
0,88,"""One day, in retrospect, the years of struggle...",18,NaN,https://mobile.twitter.com,en,1354856602885627906,Twitter Web App,1354856602885627906,NaN,lexfridman,lexfridman
1,34,"Hello,\n\nWe at Robinhood apologize for the in...",15,NaN,http://twitter.com/download/iphone,en,1354856596510277637,Twitter for iPhone,1354856596510277637,NaN,CaptainNasdaq,CaptainNasdaq
2,7,Down the hatch. https://t.co/i7CHHWU2OI,2,NaN,http://www.socialflow.com,en,1354856410501283840,SocialFlow,1354856410501283840,NaN,pcgamer,pcgamer
3,0,RT @BostonJoan: While y'all are grappling with...,5,NaN,http://twitter.com/download/iphone,en,1354856342901583872,Twitter for iPhone,1354856342901583872,NaN,techreview,techreview
4,93,House Speaker Nancy Pelosi says part of the Ca...,21,NaN,http://www.socialflow.com,en,1354856293484408834,SocialFlow,1354856293484408834,NaN,CNNPolitics,CNNPolitics


In [33]:
def extract_timeline_as_df(timeline_list):
    columns = set()
    allowed_types = [str, int]
    tweets_data = []
    for status in timeline_list:
        status_dict = dict(vars(status))
        keys = vars(status).keys()
        single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
        for k in keys:
            try:
                v_type = type(status_dict[k])
            except:
                v_type = None
            if v_type != None:
                if v_type in allowed_types:
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
        tweets_data.append(single_tweet_data)

    header_columns = list(columns)
    header_columns.append("user")
    header_columns.append("author")
    df = pd.DataFrame(tweets_data, columns=header_columns)
    return df

In [34]:
user = api.get_user("elonmusk")
user_timeline = user.timeline()
df3 = extract_timeline_as_df(user_timeline)
df3.head()

,favorite_count,in_reply_to_status_id_str,in_reply_to_user_id,text,retweet_count,in_reply_to_screen_name,source_url,in_reply_to_status_id,in_reply_to_user_id_str,lang,id,source,id_str,user,author
0,5428,1354766140606722058,1.071134e+18,@EvanKaylor Nomad,102,EvanKaylor,http://twitter.com/download/iphone,1.354766e+18,1071133717115101186,es,1354766197867368457,Twitter for iPhone,1354766197867368457,elonmusk,elonmusk
1,3582,1354177839499800578,2.519007e+09,@anbuteau 🤗,150,anbuteau,http://twitter.com/download/iphone,1.354178e+18,2519007301,und,1354761802417057793,Twitter for iPhone,1354761802417057793,elonmusk,elonmusk
2,68276,1354618772586426368,4.419640e+07,The esthetics of Cyberpunk are incredible btw....,1428,elonmusk,http://twitter.com/download/iphone,1.354619e+18,44196397,en,1354759710126338056,Twitter for iPhone,1354759710126338056,elonmusk,elonmusk
3,5617,1354598014954975236,8.322704e+08,@TrevorMahlmann @SpaceX Major esthetics improv...,182,TrevorMahlmann,http://twitter.com/download/iphone,1.354598e+18,832270448,ca,1354700252251615232,Twitter for iPhone,1354700252251615232,elonmusk,elonmusk
4,3660,1354686485681717248,1.016060e+18,@teslaownersSV @DMC_Ryan A little,160,teslaownersSV,http://twitter.com/download/iphone,1.354686e+18,1016059981907386368,en,1354686605261496320,Twitter for iPhone,1354686605261496320,elonmusk,elonmusk


##### Gamestop hype monitor
The following monitors twitter for tweets relating to the crazy war between reddit and wall street in January 2021 by checking for tweets mentioning blackberry, AMC, Nokia, and GameStop stocks.

In [35]:
query = '$BB'
BB_res = []
for i, status in enumerate(tweepy.Cursor(api.search, q=query).items(20)):
    BB_res.append(i)
    print(i, status.text, status.author.screen_name)

0 RT @AstralTrading: @arshbajwaa So far that I can see that are disabled on Robinhood - $BBBY, $AMC, $NOK, $BB, $KOSS, $EXPR, $GME. $NAKD, $A… Shitcoin12
1 Rogue One - Darth Vader Final Scene HD https://t.co/qI3R5r0Hwx via @YouTube

Can’t wait to watch all you $gme $amc… https://t.co/UBNdGqnfaO wolfvenom112
2 RT @m_k_oz: @AOC @RobinhoodApp For those that don't understand, THIS IS WHAT IS HAPPENING!!! $gme $amc $nakd $sndl $nok $bb https://t.co/KF… SimplyWreaths
3 RT @StocksToTrade: We @ STT believe ALL traders should be able to trade stocks freely/not be blocked by any brokers who partner with hedge… Rolph32345080
4 RT @NormieHuman: Hold the line!!! diamond hands!!!
$GME $DOGE $BB $BBBY $AMC $NAKD https://t.co/oBQyoYXbDy Iamalfr3do
5 RT @DraconTV: Also @TradeRepublicDE geht bei der #Gamestop Sache auch voll aufs Ganze und blockiert neben $GME auch $NOK $BB Kauf Order pau… AmarouLP
6 RT @AstralTrading: @arshbajwaa So far that I can see that are disabled on Robinhood - $BBBY, $AMC, $NOK,

In [36]:
query = '$NOK'
NOK_res = []
for i, status in enumerate(tweepy.Cursor(api.search, q=query).items(20)):
    NOK_res.append(i)
    print(i, status.text, status.author.screen_name)

0 RT @camdasilva: WELP

Can’t even buy $NOK, $GME or $AMC on Robinhood.

When the hedge funds lose, the rules are changed for the little guys… Kalique_340
1 RT @stoolpresidente: I own $amc $nok $nakd.  I bought them with the understanding we live in a free market where people can buy and sell st… clarkconquers
2 RT @camdasilva: WELP

Can’t even buy $NOK, $GME or $AMC on Robinhood.

When the hedge funds lose, the rules are changed for the little guys… jimenez__sv
3 RT @fghhgfdg: Guys someone did it! And filed a class action against @RobinhoodApp !!! NICE!!! #wallstreetbets $AMC $GME $nakd $sndl $nok ht… jesterjoe2
4 RT @emerson_chris: $AMC $GME $NOK #RobinHood https://t.co/lLzsADYAtP PAZoomer
5 RT @stoolpresidente: I own $amc $nok $nakd.  I bought them with the understanding we live in a free market where people can buy and sell st… Mainefreethink1
6 RT @Banks: Robinhood legitimately hid $NOK $AMC $GME and $NAKD from search. Internet psychology 101, this will only make things worse for 

In [37]:
query = '#AMC'
AMC_res = []
for i, status in enumerate(tweepy.Cursor(api.search, q=query, since_id=1354797040480493573).items(20)):
    AMC_res.append(i)
    print(i, status.text, status.author.screen_name)

0 RT @Ike_Saul: Robinhood is no longer allowing users to do anything but close out their #GME or #AMC positions. Hard to put into words how o… Gabechomp
1 Poor investors: "I got bills, my rent is due, got kids to feed..." #amc #gme #nok 

Robinhood: https://t.co/dGmvIJbV8C RenaylaJoye
2 “Wall Street won't be talking about GameStop in a month”. Prove the suits wrong and hold #NOK #GME #AMC #stonks https://t.co/9ntYQdMsKN KeanuTrego
3 RT @Ike_Saul: Robinhood is no longer allowing users to do anything but close out their #GME or #AMC positions. Hard to put into words how o… AnastasiaYeager
4 RT @Ike_Saul: Robinhood is no longer allowing users to do anything but close out their #GME or #AMC positions. Hard to put into words how o… LoserMcTool
5 RT @jan_wiebe: Nach #Robinhood hat nun auch #traderepublic den Handel mit u.a. #GameStop, #Nokia und #AMC gesperrt. Es geht nur noch der Ve… _frezzi
6 Disgusting behaviour today from @Trading212 and other stock apps. Market manipulation at its fines

In [39]:
query = '$GME'
gamestop_res = []
for i, status in enumerate(tweepy.Cursor(api.search, q=query).items(20)):
    BB_res.append(i)
    print(i, status.text, status.author.screen_name)

0 RT @hello__caitlin: The stuff going on with $GME, Robinhood and Wall Street is a perfect, chef’s-kiss, compact case study in how our system… GrandmasHGArt
1 RT @justinkan: Just got a tip that Citadel reloaded their shorts before they told Robinhood to stop trading $GME.

If this is true, Ken Gri… atharvafulay
2 Silver short squeeze underway $AMC $GME $SLV
https://t.co/f2cwSfhQlW TheHigherSpace
3 RT @AstralTrading: @arshbajwaa So far that I can see that are disabled on Robinhood - $BBBY, $AMC, $NOK, $BB, $KOSS, $EXPR, $GME. $NAKD, $A… SB_Airon
4 I just thought this was interesting considering $GME  &amp; other securities in the last few days.… https://t.co/N4TrEpKiuv dmosleymanageme
5 RT @themaxburns: The lesson of WSB's $GME $NOK $BB $AMC gambit is that investors aren't winners because they find the best way to profit un… jmere09nyc
6 RT @AstralTrading: @chamath Updated List of Disabled #stock Tickers

Robinhood - $BBBY, $AMC, $NOK, $BB, $KOSS, $EXPR, $GME. $NAKD, $AAL, $… hangryhome